# Scraping Dynamic Content: Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import tqdm
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from lxml import etree
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time

In [4]:
chrome_path = 'C:/chromedriver'
ser = Service(chrome_path)
webdriver.Chrome(service=ser)
driver = webdriver.Chrome(service=ser)

In [ ]:
#driver.close()
#driver.quit()

In [5]:
url = 'https://www.ibge.gov.br/apps/populacao/projecao/index.html?utm_source=portal&utm_medium=popclock&utm_campaign=novo_popclock'
driver.get(url)
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html,'lxml')
driver.maximize_window()

In [ ]:
state = driver.find_element(by='id',value = 'seletor_de_uf_lista').text
state_list = state.split(' ')

# Transform Data

In [ ]:
def generate_pd_dados(sigla_uf,arquivo_name,i):
    '''This function generate general data from IBGE by state'''
    df_dados = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=[0,1], nrows = 51)
    try:        
        df_dados = df_dados.drop(columns=['Unnamed: 20'])
    except:
        df_dados = df_dados.drop(columns=['Unnamed: 21'])       
    df_dados['UF']= sigla_uf
    return(df_dados)#    

def generate_pd_total(sigla_uf,arquivo_name,i):
    '''This function generate total age group data by state'''
    df_dados_total = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=56, nrows = 20)
    df_dados_total = df_dados_total.drop(columns=['Unnamed: 52'])       
    df_dados_total['Tipo']='Total'
    df_dados_total['UF']= sigla_uf
    return(df_dados_total)#    

def generate_pd_homem(sigla_uf,arquivo_name,i):
    '''This function generate male age group data by state'''
    df_dados_homem = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=78, nrows = 20)
    df_dados_homem = df_dados_homem.drop(columns=['Unnamed: 52'])      
    df_dados_homem['Tipo']='Homem'
    df_dados_homem['UF']= sigla_uf
    return(df_dados_homem)#    

def generate_pd_mulher(sigla_uf,arquivo_name,i):
    '''This function generate female age group data by state'''
    df_dados_mulher = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=100, nrows = 20)
    df_dados_mulher = df_dados_mulher.drop(columns=['Unnamed: 52'])
    df_dados_mulher['Tipo']='Mulher'
    df_dados_mulher['UF']= sigla_uf
    return(df_dados_mulher)#    

def generate_pd_dados_brasil(sigla_uf,arquivo_name,i):
    '''This function generate consolidated general data from IBGE (country)'''
    df_dados_brasil = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=123, nrows = 51)
    df_dados_brasil = df_dados_brasil.drop(columns=['Unnamed: 20'])
    df_dados_brasil['UF']= 'BR'
    return(df_dados_brasil)#    

def generate_pd_dados_fet_brasil(sigla_uf,arquivo_name,i):
    '''This function generate consolidated total age group (country)'''
    df_dados_fet_brasil = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=177, nrows = 20)
    df_dados_fet_brasil = df_dados_fet_brasil.drop(columns=['Unnamed: 52'])
    df_dados_fet_brasil['Tipo']='Total'
    df_dados_fet_brasil['UF']= 'BR'
    return(df_dados_fet_brasil)#    

def generate_pd_dados_h_brasil(sigla_uf,arquivo_name,i):
    '''This function generate consolidated male age group (country)'''
    df_dados_h_brasil = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=199, nrows = 20)
    df_dados_h_brasil = df_dados_h_brasil.drop(columns=['Unnamed: 52'])
    df_dados_h_brasil['Tipo']='Homem'
    df_dados_h_brasil['UF']= 'BR'
    return(df_dados_h_brasil)#    

def generate_pd_dados_m_brasil(sigla_uf,arquivo_name,i):
    '''This function generate consolidated female age group (country)'''
    df_dados_m_brasil = pd.read_csv(f'{arquivo_name}', encoding = 'latin1', sep=';', skiprows=221, nrows = 20)
    df_dados_m_brasil = df_dados_m_brasil.drop(columns=['Unnamed: 52'])
    df_dados_m_brasil['Tipo']='Mulher'
    df_dados_m_brasil['UF']= 'BR'
    return(df_dados_m_brasil)#    

# EXTRACTION: Get Data from IBGE

In [ ]:
sigla_uf = ''
arquivo_name = ''
for i in range(len(state_list)):
    if i>20:
        sigla_uf = state_list[i]
        i += 1
        print(i)
        driver.find_element('xpath',f'//*[@id="seletor_de_uf_lista"]/a[{i}]').click()
        time.sleep(2.5)
        driver.switch_to.frame(driver.find_element('id','gx_csv'))
        time.sleep(1.5)
        driver.find_element('xpath','/html/body/div[4]/div/textarea').click()  
        time.sleep(1.5)
        a = ActionChains(driver)    
        a.key_down(Keys.CONTROL).send_keys('C').key_up(Keys.CONTROL).perform()  
        time.sleep(1.5)
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get("https://forms.gle/JJ6DcU8qkS2EVuTF8")
        time.sleep(1.5)
        driver.find_element('xpath','/html/body/div/div[2]/form/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div[2]/textarea').click()
        a.key_down(Keys.CONTROL).send_keys('V').key_up(Keys.CONTROL).perform()
        time.sleep(6.0)
        arquivo = driver.find_element('xpath','/html/body/div/div[2]/form/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div[2]/textarea').get_attribute('data-initial-value')
        time.sleep(2.5)
        text_file = open(f"arq{i}.csv", "w")
        text_file.write(arquivo)
        text_file.close()
        arquivo_name = text_file.name
        print(arquivo_name)
        driver.close()
        time.sleep(2.0)
        driver.switch_to.window(driver.window_handles[0])
        # Transform data by state
        df_dados_gerais_aux = generate_pd_dados(sigla_uf,arquivo_name,i)
        if i==1:
            df_dados_gerais = df_dados_gerais_aux
        else: 
            df_dados_gerais= pd.concat([df_dados_gerais,df_dados_gerais_aux])

        df_dados_total_hm_aux = generate_pd_total(sigla_uf,arquivo_name,i)
        if i==1:
            df_dados_faixa_etaria = df_dados_total_hm_aux
        else: 
            df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_total_hm_aux])
    
        df_dados_homem_aux = generate_pd_homem(sigla_uf,arquivo_name,i)
        df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_homem_aux]) 

        df_dados_mulher_aux = generate_pd_mulher(sigla_uf,arquivo_name,i)
        df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_mulher_aux]) 
        # Consolidated data - only 1 is necessary (This information exist at the end of each state)
        if i==1:
            df_dados_brasil = generate_pd_dados_brasil(sigla_uf,arquivo_name,i)
            df_dados_gerais= pd.concat([df_dados_gerais,df_dados_brasil])
            
            df_dados_fet_brasil = generate_pd_dados_fet_brasil(sigla_uf,arquivo_name,i)
            df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_fet_brasil])
            
            df_dados_h_brasil = generate_pd_dados_h_brasil(sigla_uf,arquivo_name,i)
            df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_h_brasil])
            
            df_dados_m_brasil= generate_pd_dados_m_brasil(sigla_uf,arquivo_name,i)
            df_dados_faixa_etaria = pd.concat([df_dados_faixa_etaria,df_dados_m_brasil])
            
        time.sleep(2.5)


# Load Data

In [ ]:
df_dados_faixa_etaria.to_csv('df_dados_faixa_etaria.csv',index=False, sep=';')
df_dados_gerais.to_csv('df_dados_gerais.csv',index=False, sep=';')